In [ ]:
pip install web3

In [ ]:
pip install multicall

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import time
from datetime import datetime
import requests
import json
import pandas as pd
from web3 import Web3
from web3.exceptions import (InvalidEventABI, LogTopicError, MismatchedABI)
from web3._utils.events import get_event_data
from eth_utils import (encode_hex, event_abi_to_log_topic)
from web3.datastructures import AttributeDict
from multicall import Call, Multicall
from IPython.core.display import display, HTML

Infura_Project_ID = 'Infura_Project_ID' # https://infura.io/docs
assert Infura_Project_ID != 'Infura_Project_ID', "Infura Project ID required!"
Etherscan_APIKEY   = None #optional, but recommended
GearboxAddressProvider = '0xcF64698AFF7E5f27A11dff868AF228653ba53be0'

w3_eth = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/'+Infura_Project_ID, request_kwargs={'timeout': 10}))
print ('Ethereum connected:', w3_eth.isConnected())


Ethereum connected: True


In [2]:
def parse_abi(abi_dict, abi_type = None):
    recs = []
    for contract_type in abi_dict.keys():
        for rec in [x for x in abi_dict[contract_type] if not abi_type or x['type'] == abi_type]:
            topic = None
            if rec['type'] == 'event':
                topic = encode_hex(event_abi_to_log_topic(rec))
            name = None
            if 'name' in rec:
                name = rec['name']
                
            recs.append({'contract_type': contract_type,
                           'name'        : name,
                           'type'        : rec['type'],
                           'abi'         : rec,
                           'topic'       : topic,
                           })
    
    df = pd.DataFrame(recs)
    return df
def get_logs (w3, contract_address, topics=None, from_block=0, to_block='latest'):
    logs = w3.eth.get_logs({"address": contract_address
                           ,"topics":topics 
                           ,"fromBlock": from_block 
                           ,"toBlock": to_block
                           })

    all_events = []
    for l in logs:
        try:
            evt_topic0 = l['topics'][0].hex()
            evt_abi = df_abi[df_abi['topic']== evt_topic0]['abi'].values[0] 
            evt = get_event_data(w3.codec, evt_abi, l)
        except MismatchedABI: #if for some reason there are other events 
            pass
        all_events.append(evt)
    df = pd.DataFrame(all_events)
    return df

In [3]:
def pull_abi_etherscan(contract_address, apikey = Etherscan_APIKEY):
    url = 'https://api.etherscan.io/api?module=contract&action=getabi'
    params = {'address':contract_address,'apikey' : apikey}
    
    if not apikey:
        time.sleep(5) # rate-limit when apikey is empty
        
    response = requests.get(url, params=params)
    response_json = json.loads(response.text)
    
    if response_json['status']  == '1':
        return json.loads(response_json['result'])
    else:
        raise Exception(response_json['result'])

In [4]:
AddressProvider = Web3.toChecksumAddress(GearboxAddressProvider)
AddressProvider_abi = pull_abi_etherscan(AddressProvider)

AccountFactory = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getAccountFactory().call()
print('AccountFactory:', AccountFactory)
AccountFactory_abi = pull_abi_etherscan(AccountFactory)
countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccounts().call()
print('countCreditAccounts:', countCreditAccounts)
masterCreditAccount = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.masterCreditAccount().call()
print('masterCreditAccount:', masterCreditAccount)
CreditAccount_abi = pull_abi_etherscan(masterCreditAccount)

DataCompressor = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getDataCompressor().call()
print('DataCompressor:', DataCompressor)
DataCompressor_abi = pull_abi_etherscan(DataCompressor)

ContractsRegister = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getContractsRegister().call()
print('ContractsRegister:', ContractsRegister)
ContractsRegister_abi = pull_abi_etherscan(ContractsRegister)

CreditManagers = w3_eth.eth.contract(address=ContractsRegister, abi=ContractsRegister_abi).functions.getCreditManagers().call()
print('CreditManagers:', CreditManagers)

cm_dict = {AccountFactory: {'symbol': None, 'decimals': None}}
allowedTokens = {}
for i, CreditManager in enumerate(CreditManagers):
    if i==0:
        CreditManager_abi = pull_abi_etherscan(CreditManager)

    Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlyingToken().call()  
    if i==0:
        Token_abi = pull_abi_etherscan(Token)   
    
    CreditFilter = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFilter().call()
    if i==0:
        CreditFilter_abi = pull_abi_etherscan(CreditFilter)      
    allowedTokensCount = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokensCount().call()
    
    cm_dict[CreditManager] = {'symbol': w3_eth.eth.contract(address=Token, abi=Token_abi).functions.symbol().call()
                              ,'decimals': w3_eth.eth.contract(address=Token, abi=Token_abi).functions.decimals().call()
                              ,'CreditFilter': CreditFilter
                              ,'allowedTokensCount': allowedTokensCount
                              ,'allowedTokens':{}
                             }
    for token_id in range(allowedTokensCount):
        allowed_token = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokens(token_id).call()
        allowed_token_symbol = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.symbol().call()
        allowed_token_decimals = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.decimals().call()
        #cm_dict[CreditManager]['allowedTokens'].update({allowed_token: {'symbol': allowed_token_symbol, 'decimals':allowed_token_decimals}})  
        
        allowedTokens[allowed_token]={'symbol': allowed_token_symbol, 'decimals':allowed_token_decimals}
    
df_abi = parse_abi({'AddressProvider':AddressProvider_abi, 
                    'AccountFactory': AccountFactory_abi, 
                    'DataCompressor': DataCompressor_abi,
                    'CreditManager': CreditManager_abi,
                    'CreditFilter': CreditFilter_abi,
                    'CreditAccount': CreditAccount_abi,
                   })

display(cm_dict) 
display(allowedTokens)

AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
countCreditAccounts: 5001
masterCreditAccount: 0x373A292b93ff9017D28E64154ef83b99D5C4e270
DataCompressor: 0x0050b1ABD1DD2D9b01ce954E663ff3DbCa9193B1
ContractsRegister: 0xA50d4E7D8946a7c90652339CDBd262c375d54D99
CreditManagers: ['0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB', '0x2664cc24CBAd28749B3Dd6fC97A6B402484De527', '0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0', '0xC38478B0A4bAFE964C3526EEFF534d70E1E09017']


{'0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04': {'symbol': None,
  'decimals': None},
 '0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB': {'symbol': 'DAI',
  'decimals': 18,
  'CreditFilter': '0x948D33a9537cf13bCC656218B385D19E5b6693e8',
  'allowedTokensCount': 15,
  'allowedTokens': {}},
 '0x2664cc24CBAd28749B3Dd6fC97A6B402484De527': {'symbol': 'USDC',
  'decimals': 6,
  'CreditFilter': '0x301E7Ed8ac816747A65cf67D8901659e637a4383',
  'allowedTokensCount': 15,
  'allowedTokens': {}},
 '0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0': {'symbol': 'WETH',
  'decimals': 18,
  'CreditFilter': '0x790D170C12e62aDa6CD2D409a50177680c4ddF29',
  'allowedTokensCount': 15,
  'allowedTokens': {}},
 '0xC38478B0A4bAFE964C3526EEFF534d70E1E09017': {'symbol': 'WBTC',
  'decimals': 8,
  'CreditFilter': '0xcF223eB26dA2Bf147D01b750d2D2393025cEA7Ca',
  'allowedTokensCount': 15,
  'allowedTokens': {}}}

{'0x6B175474E89094C44Da98b954EedeAC495271d0F': {'symbol': 'DAI',
  'decimals': 18},
 '0x111111111117dC0aa78b770fA6A738034120C302': {'symbol': '1INCH',
  'decimals': 18},
 '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9': {'symbol': 'AAVE',
  'decimals': 18},
 '0xc00e94Cb662C3520282E6f5717214004A7f26888': {'symbol': 'COMP',
  'decimals': 18},
 '0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b': {'symbol': 'DPI',
  'decimals': 18},
 '0x956F47F50A910163D8BF957Cf5846D573E7f87CA': {'symbol': 'FEI',
  'decimals': 18},
 '0x514910771AF9Ca656af840dff83E8264EcF986CA': {'symbol': 'LINK',
  'decimals': 18},
 '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F': {'symbol': 'SNX',
  'decimals': 18},
 '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984': {'symbol': 'UNI',
  'decimals': 18},
 '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48': {'symbol': 'USDC',
  'decimals': 6},
 '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599': {'symbol': 'WBTC',
  'decimals': 8},
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': {'symbol': 'WETH',
  'de

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
def combine_description(description):
    text_description = ''
    for x in description:
        if x['type']=='tuple': 
            text_description += '(' + combine_description(x['components'])+')'+','
        elif x['type']=='tuple[]': 
            text_description += '(' + combine_description(x['components']) + ')[]' +','
            
        else:
            text_description += x['internalType']+',' 
    return text_description[0:-1]      

def get_function_signature(function_name, df_abi, inputs=None, outputs=None):
    if not inputs:
        inputs = df_abi[df_abi['name']==function_name]['abi'].values[0]['inputs']
    if not outputs:
        outputs= df_abi[df_abi['name']==function_name]['abi'].values[0]['outputs']
    
    return function_name +'(' + combine_description(inputs) + ')' + '(' + combine_description(outputs) +')' 

def get_data_multicall(df, function_name, df_abi, contract_address = None):
    
    function_signature = get_function_signature(function_name, df_abi )
    #print(function_signature)
    
    if function_name == 'creditAccounts':
        multi_result = Multicall([
                    Call(contract_address, [function_signature, x], [[x, Web3.toChecksumAddress]]) for x in df['id']
                    ]
                    ,_w3 = w3_eth)
    elif function_name == 'creditManager':
        multi_result = Multicall([
                        Call(y, [function_signature], [[x, Web3.toChecksumAddress]]) for x,y in zip(df['id'],df['CA'])
                        ]
                        ,_w3 = w3_eth) 
    elif function_name == 'borrowedAmount':
        multi_result = Multicall([
                        Call(y, [function_signature], [[x, None]]) for x,y in zip(df['id'],df['CA'])
                        ]
                        ,_w3 = w3_eth) 
    elif function_name == 'since':
        multi_result = Multicall([
                        Call(y, [function_signature], [[x, None]]) for x,y in zip(df['id'],df['CA'])
                        ]
                        ,_w3 = w3_eth)
    elif function_name == 'getCreditAccountDataExtended':
        #print(function_signature)
        multi_result = Multicall([
                        Call(contract_address, [function_signature, y, z], [[x, None]]) for x,y,z in zip(df['id'],df['CM'],df['Borrower'])
                        ]
                        ,_w3 = w3_eth)    
    try:
        multi_result = multi_result()
    except (requests.exceptions.HTTPError):
        time.sleep(3)
        multi_result = multi_result()
        
    d_multi_result = AttributeDict.recursive(multi_result)
    return d_multi_result


In [13]:
df = pd.DataFrame()
df['id'] = range(countCreditAccounts)
for ids in list(chunks(list(df['id']), 400)): #chunk size for multicall = 400
    id_range = list(df['id'].isin(ids))
    d_ca = get_data_multicall(df.loc[id_range], 'creditAccounts', df_abi, AccountFactory)
    df.loc[id_range,'CA'] = df.loc[id_range].apply(lambda x: d_ca[x['id']], axis=1)

    d_cm = get_data_multicall(df.loc[id_range], 'creditManager', df_abi)
    df.loc[id_range,'CM'] = df.loc[id_range].apply(lambda x: d_cm[x['id']], axis=1)
    
    d_amount = get_data_multicall(df.loc[id_range], 'borrowedAmount', df_abi)
    df.loc[id_range,'borrowedAmount'] = df.loc[id_range].apply(lambda x: d_amount[x['id']] if d_amount[x['id']] > 1 else 0 , axis=1)
    
    d_since = get_data_multicall(df.loc[id_range], 'since', df_abi)
    df.loc[id_range,'Since'] = df.loc[id_range].apply(lambda x: d_since[x['id']], axis=1)

df['Since'] = df['Since'].astype(int)
df['Decimals'] = df.apply(lambda x: cm_dict[x['CM']]['decimals'], axis=1)
df['Symbol'] = df.apply(lambda x: cm_dict[x['CM']]['symbol'] , axis=1)
df

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,600000000000000000,13864162,18.0,WETH
...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None


In [14]:
#Open and Close
OpenCreditAccount_topic  =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic =  df_abi[(df_abi['name']=='CloseCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic =  df_abi[(df_abi['name']=='RepayCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]

print('OpenCreditAccount_topic:', OpenCreditAccount_topic)
print('CloseCreditAccount_topic:', CloseCreditAccount_topic)
print('RepayCreditAccount_topic:', RepayCreditAccount_topic)

for row in df.loc[df['CM']!=AccountFactory].itertuples():
    logs = get_logs(w3_eth, row.CM, [[OpenCreditAccount_topic, CloseCreditAccount_topic, RepayCreditAccount_topic]], row.Since, 'latest')
    
    open_events = logs[logs['event'] =='OpenCreditAccount']['args'].values 
    close_events = logs[logs['event'] =='CloseCreditAccount']['args'].values 
    repay_events = logs[logs['event'] =='RepayCreditAccount']['args'].values
    
    CA_open_event = [x for x in open_events if x['creditAccount']== row.CA] # Open CA
    if len(CA_open_event) > 0:
        Borrower = CA_open_event[0]['onBehalfOf']
        df.loc[df['id']==row.id, 'Borrower'] = Borrower
    
        CA_close_event = [x for x in close_events if x['to']== Borrower] # Close CA
        if len(CA_close_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'Amount', 'Amount_Decimals']] = [None, 0, 0]

        CA_repay_event = [x for x in repay_events if x['to']== Borrower] # Repay CA
        if len(CA_repay_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'Amount', 'Amount_Decimals']] = [None, 0, 0]
            
    if row.id % 50 == 0:
        print (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()), ':', row.id)

display(df)    

OpenCreditAccount_topic: 0x7b20ae77867a263a1074203a2da261ef0d096c99395c59c9d4a0104b9f334a27
CloseCreditAccount_topic: 0xca05b632388199c23de1352b2e96fd72a0ec71611683330b38060c004bbf0a76
RepayCreditAccount_topic: 0xe7c7987373a0cc4913d307f23ab8ef02e0333a2af445065e2ef7636cffc6daa7
2021-12-27 20:02:38 : 0
2021-12-27 20:02:52 : 50
2021-12-27 20:03:05 : 100


,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,Amount,Amount_Decimals
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI,0x8a4B89D76A1a745a4A1aDEBd3793253FBa0adADc,NaN,NaN
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH,None,0.0,0.0
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI,None,0.0,0.0
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH,None,0.0,0.0
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,600000000000000000,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,NaN,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,NaN,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,NaN,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,NaN,NaN,NaN


In [15]:
data_cols = [x['name'] for x in df_abi[df_abi['name']=='getCreditAccountDataExtended']['abi'].values[0]['outputs'][0]['components']]

df['batchtime'] = datetime.now()

for ids in list(chunks(list(df[pd.notna(df['Borrower'])].loc[:,'id']), 400)): #chunk size for multicall = 400
    id_range = list(df['id'].isin(ids))
    d_data = get_data_multicall(df.loc[id_range], 'getCreditAccountDataExtended', df_abi, DataCompressor)
    
    data_to_frame = ['borrowedAmount',
                     'borrowedAmountPlusInterest',
                     'totalValue',
                     'healthFactor',
                     'borrowRate',
                     'repayAmount',
                     'liquidationAmount',
                     'canBeClosed',
                    ]
    for data in data_to_frame:
        df.loc[id_range, data] = df.loc[id_range].apply(lambda x: {y:z for y, z in zip(data_cols, d_data[x['id']])}[data]
                                                        , axis=1)
        
    tokens_to_frame = [allowedTokens[x]['symbol'] for x in allowedTokens] 
    for token in tokens_to_frame:
        df.loc[id_range, 'Balance_'+token] = df.loc[id_range].apply(lambda x: {allowedTokens[Web3.toChecksumAddress(d[0])]['symbol']:d[1] for d in list({y:z for y, z in zip(data_cols, d_data[x['id']])}['balances'])}[token]#{y:z for y, z in zip(data_cols, d_data[x['id']])}[data]
                                                        , axis=1)
    
        
df 

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,Amount,Amount_Decimals,...,Balance_FEI,Balance_LINK,Balance_SNX,Balance_UNI,Balance_USDC,Balance_WBTC,Balance_WETH,Balance_YFI,Balance_yvDAI,Balance_yvUSDC
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI,0x8a4B89D76A1a745a4A1aDEBd3793253FBa0adADc,NaN,NaN,...,0,0.0,0.0,0,0.0,0.0,7.325157e+17,0.0,0.0,0.0
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH,None,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI,None,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH,None,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,600000000000000000,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,NaN,NaN,...,0,0.0,0.0,0,0.0,0.0,9.000000e+17,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df[pd.notna(df['Borrower'])] # active CAs

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,Amount,Amount_Decimals,...,Balance_FEI,Balance_LINK,Balance_SNX,Balance_UNI,Balance_USDC,Balance_WBTC,Balance_WETH,Balance_YFI,Balance_yvDAI,Balance_yvUSDC
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI,0x8a4B89D76A1a745a4A1aDEBd3793253FBa0adADc,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,7.325157e+17,0.0,0.0,0.0
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,600000000000000000,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,9.000000e+17,0.0,0.0,0.0
7,7,0x796384Bd0566B43163c2f5094FD54Ed9B3dA1Ef1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13864650,18.0,WETH,0xCf7841FD47bF33004B07eDE874e7aFC714616231,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,1.200000e+18,0.0,0.0,0.0
8,8,0xF91027C04807c71c7771a2314A622dFA0fE509A1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13864789,18.0,WETH,0x896b94f4f27f12369698C302e2049cAe86936BbB,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,2.000000e+18,0.0,0.0,0.0
11,11,0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,290000000000000000,13864924,18.0,WETH,0xC16414AC1fedfDAC4F8A09674D994e1BbB9d7113,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,1.290000e+18,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,0x54046d55aF25d6689C35b5bC9e3b09332aD0662e,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,3000000000000000000,13887429,18.0,WETH,0x0D72A35DF86eFdefE3593b50BbD9EdecFA5BAAD7,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,4.000000e+18,0.0,0.0,0.0
99,99,0xFCfBf5eDB1948Ccb0a412DE53f1A4FA825d702d7,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,3000000000,13887429,6.0,USDC,0x2aE1BCBaa70ce157cAF7614E2c6246BfB950321e,NaN,NaN,...,0,0.0,0.0,0,4.000000e+09,0.0,0.000000e+00,0.0,0.0,0.0
100,100,0xbD18023746e09Cf17f3e832003Be9264eaBD8612,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,3000000000000000000,13887461,18.0,WETH,0x01d104FFE610Ac60a22628C4558391d1CB55aA6c,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,4.000000e+18,0.0,0.0,0.0
101,101,0x934BE1e79E3D081429855Afd09d2d94aaA504770,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,3000000000000000000,13887473,18.0,WETH,0xdDA32e049c6555BA701246618f65866a679AD168,NaN,NaN,...,0,0.0,0.0,0,0.000000e+00,0.0,4.000000e+18,0.0,0.0,0.0


In [17]:
non_number_cols = [x for x in df.columns if x not in ['CA', 'CM' ,'Symbol', 'Borrower', 'batchtime', 'canBeClosed']]
df[non_number_cols] = df[non_number_cols].astype('float64')
df['canBeClosed'] =  df['canBeClosed'].astype('bool')

df.dtypes

id                                   float64
CA                                    object
CM                                    object
borrowedAmount                       float64
Since                                float64
Decimals                             float64
Symbol                                object
Borrower                              object
Amount                               float64
Amount_Decimals                      float64
batchtime                     datetime64[ns]
borrowedAmountPlusInterest           float64
totalValue                           float64
healthFactor                         float64
borrowRate                           float64
repayAmount                          float64
liquidationAmount                    float64
canBeClosed                             bool
Balance_DAI                          float64
Balance_1INCH                        float64
Balance_AAVE                         float64
Balance_COMP                         float64
Balance_DP

Data Sduto Report
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0
